# Credits
Code adapted from: https://github.com/decentralion/tf-dev-summit-tensorboard-tutorial/blob/master/mnist.py

In [1]:
import os
import os.path
import shutil
import tensorflow as tf

LOGDIR = "./mnist_demo/"
### MNIST EMBEDDINGS ###
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(train_dir=LOGDIR + "data", one_hot=True)

/Users/trungtv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist_demo/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist_demo/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as 

In [2]:
def conv_layer(input, size_in, size_out, name='conv'):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([5, 5, size_in, size_out]), name='W')
        b = tf.Variable(tf.zeros([size_out]), name='B')
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def fc_layer(input, size_in, size_out, name='fc'):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([size_in, size_out]), name='W')
        b = tf.Variable(tf.zeros([size_out]), name='B')
        act = tf.matmul(input, w) + b
        return act

In [3]:
tf.reset_default_graph()
sess = tf.Session()

In [4]:
def mnist_model(learning_rate):

    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # create the network 
    # first convec layer
    conv1 = conv_layer(x_image, 1, 32, "conv1")
    # second convec layer
    conv_out = conv_layer(conv1, 32, 64, "conv2")
    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])
    fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, 'fc1')
    logits = fc_layer(fc1, 1024, 10, 'fc2')
    
    #compute cross entropy as our loss function 
    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                logits=logits, labels=y))
    # use and AdapOptimizer to train the network
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)
    
    # compute the accuracy
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Initialize all the variables 
    sess.run(tf.global_variables_initializer())
    
    # Train for 2000 steps
    for i in range(2000):
        batch = mnist.train.next_batch(100)
        # Occasionally report accuracy 
        if i % 500 == 0:
            [train_accuracy] = sess.run([accuracy], feed_dict={x: batch[0], y: batch[1]})
            print("step %d, training accuracy %g" % (i, train_accuracy))
            
        # Run the training step
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

In [ ]:
mnist_model(1e-4)

step 0, training accuracy 0.09


step 0, training accuracy 0.1  
step 500, training accuracy 0.08  
step 1000, training accuracy 0.12


## Let visualize the Tensorflow graph
tensorboard --logdir ./mnist_demo/2

In [6]:
writer = tf.summary.FileWriter("./mnist_demo/2")
writer.add_graph(sess.graph)